In [1]:
# import sys
# sys.path
# sys.path.insert(0, '/Users/zhengzezhou/scikit-learn')

In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from tqdm import tqdm

from sklearn.utils import shuffle


In [59]:
def predict_var(self, X, method = "corrected"):

    predict_all = np.zeros((X.shape[0], self.n_estimators))
    for t_idx in range(self.n_estimators):
        predict_all[:, t_idx] = self.estimators_[t_idx].predict(X)
    pred = np.mean(predict_all, axis = 1)

    inbag_times_ = self.inbag_times_

    m = X.shape[0]

    if method == "BM":
        cond_exp_full = np.zeros((self.n_samples_, m))

        for i in range(self.n_samples_):
            # cond_exp_full[i, :] = np.mean(predict_all[:, inbag_times_[i, :] >= 1], axis=1)
            cond_exp_full[i, :] = np.average(predict_all, weights = inbag_times_[i, :], axis = 1)

        zeta1_full = np.zeros(m)
        zetan_full = np.zeros(m)
        variance = np.zeros(m)
        for i in range(m):
            zeta1_full[i] = np.var(cond_exp_full[:, i])
            zetan_full[i] = np.var(predict_all[i, :])
        variance = zeta1_full * (self.n_subsamples_ ** 2) / self.n_samples_ + zetan_full / self.n_estimators
        
        return [float(variance), float(zeta1_full), float(zetan_full)]
        
    elif method == "IJ":
        
        f_centered = predict_all - np.mean(predict_all, axis=1).reshape(m, 1)
        i_centered = inbag_times_ - np.mean(inbag_times_, axis=1).reshape(self.n_samples_, 1)
        corr = np.dot(f_centered, i_centered.T) / self.n_estimators
        cov = np.dot(corr, corr.T)
        zetan_full = np.cov(predict_all)
        covariance = cov + zetan_full / self.n_estimators

        return [float(np.diagonal(covariance)), float(cov), float(zetan_full)]
    
    elif method == "corrected":
        
        cond_exp_full = np.zeros((self.n_samples_, m))

        for i in range(self.n_samples_):
            # cond_exp_full[i, :] = np.mean(predict_all[:, inbag_times_[i, :] >= 1], axis=1)
            cond_exp_full[i, :] = np.average(predict_all, weights = inbag_times_[i, :], axis = 1)

        inbag_times_ = inbag_times_[np.sum(inbag_times_, axis = 1) > 0, ]

        nk = np.sum(inbag_times_, axis = 1)
        K = len(nk)
        C = np.sum(nk)     

        SSr = np.dot(((cond_exp_full - pred) ** 2).T, nk)

        SSe = [0] * m

        for i in range(K):
            SSei = np.sum((predict_all[:, inbag_times_[i, :]>=1].T - cond_exp_full[i, :]) ** 2, axis = 0)
            SSe += SSei

        sigma_e_squared = SSe / (C - K)

        sigma_M_squared = (SSr - (K - 1) * sigma_e_squared) / (C - np.sum(nk ** 2) / C)
        
        sigma_M_squared_without = SSr / C
        
        return  [(self.n_subsamples_ ** 2) / self.n_samples_ * float(sigma_M_squared), (self.n_subsamples_ ** 2) / self.n_samples_ * float(sigma_M_squared_without)]
        

In [53]:
df = pd.read_csv("CASP.csv")

In [54]:
df.head()

,RMSD,F1,F2,F3,F4,F5,F6,F7,F8,F9
0,17.284,13558.30,4305.35,0.31754,162.1730,1.872791e+06,215.3590,4287.87,102,27.0302
1,6.021,6191.96,1623.16,0.26213,53.3894,8.034467e+05,87.2024,3328.91,39,38.5468
2,9.275,7725.98,1726.28,0.22343,67.2887,1.075648e+06,81.7913,2981.04,29,38.8119
3,15.851,8424.58,2368.25,0.28111,67.8325,1.210472e+06,109.4390,3248.22,70,39.0651
4,7.962,7460.84,1736.94,0.23280,52.4123,1.021020e+06,94.5234,2814.42,41,39.9147


In [56]:
df = shuffle(df)
df.head()

,RMSD,F1,F2,F3,F4,F5,F6,F7,F8,F9
6856,13.612,5962.60,2053.84,0.34445,59.4506,8.444319e+05,81.9593,2519.02,64,41.4304
32367,14.711,7189.79,2575.24,0.35818,66.8440,1.028778e+06,99.7303,3612.59,79,38.0773
26510,1.721,14732.40,4370.39,0.29665,176.6870,2.059394e+06,238.1280,5363.64,175,24.6951
25291,1.496,7128.70,1825.45,0.25607,82.2566,9.885535e+05,111.3400,3739.88,19,46.5464
4266,3.928,14157.20,4842.67,0.34206,161.2610,2.011370e+06,232.6270,4515.30,64,29.4748


In [57]:
df.describe()

,RMSD,F1,F2,F3,F4,F5,F6,F7,F8,F9
count,45730.000000,45730.000000,45730.000000,45730.000000,45730.000000,4.573000e+04,45730.000000,45730.000000,45730.000000,45730.000000
mean,7.748528,9871.596995,3017.367175,0.302392,103.492433,1.368299e+06,145.638061,3989.755990,69.975071,34.523664
std,6.118312,4058.138034,1464.324663,0.062886,55.424985,5.640367e+05,69.999230,1993.574575,56.493443,5.979755
min,0.000000,2392.050000,403.500000,0.092500,10.310100,3.194902e+05,31.970400,0.000000,0.000000,15.228000
25%,2.305000,6936.680000,1979.045000,0.258740,63.563900,9.535912e+05,94.757500,3165.322500,31.000000,30.424725
50%,5.030000,8898.805000,2668.155000,0.300150,87.740800,1.237219e+06,126.176000,3840.170000,54.000000,35.299300
75%,13.379000,12126.150000,3786.410000,0.342890,133.646750,1.690920e+06,181.468500,4644.192500,91.000000,38.870800
max,20.999000,40034.900000,15312.000000,0.577690,369.317000,5.472011e+06,598.408000,105948.170000,350.000000,55.300900


In [58]:
df_train = df[:45000]
df_test = df[45000:]

## k = 100

In [146]:
result_100 = {500:  {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []},
              1000: {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []},
              2500: {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []},
              5000: {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []}}

rep = 45
incre = 1000
n_sub = 100

for _ in tqdm(range(20)):
    
    test = df_test.iloc[np.random.choice(730)].values[1:].reshape(1, -1)

    # print("========================= B = 500 =========================")

    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 500)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_100[500]["normal"].append(stats.normaltest(pred))
    result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_100[500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_100[500]["cc_c"].append(count_c / rep)
    result_100[500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 1000 =========================")
    
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 1000)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        

    result_100[1000]["normal"].append(stats.normaltest(pred))
    result_100[1000]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_100[1000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_100[1000]["cc_c"].append(count_c / rep)
    result_100[1000]["cc_uc"].append(count_uc / rep)



    # print("========================= B = 2500 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 2500)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])


    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        

    result_100[2500]["normal"].append(stats.normaltest(pred))
    result_100[2500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_100[2500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_100[2500]["cc_c"].append(count_c / rep)
    result_100[2500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 5000 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 5000)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1      
            
    result_100[5000]["normal"].append(stats.normaltest(pred))
    result_100[5000]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_100[5000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_100[5000]["cc_c"].append(count_c / rep)
    result_100[5000]["cc_uc"].append(count_uc / rep)



 40%|████      | 8/20 [2:03:49<3:17:44, 988.68s/it] <ipython-input-146-fe9ebcac4b1b>:36: RuntimeWarning: invalid value encountered in sqrt
  lower = pred[r] - 1.96 * np.sqrt(var_c[r])
<ipython-input-146-fe9ebcac4b1b>:37: RuntimeWarning: invalid value encountered in sqrt
  upper = pred[r] + 1.96 * np.sqrt(var_c[r])
 50%|█████     | 10/20 [2:29:08<2:25:19, 871.97s/it]<ipython-input-146-fe9ebcac4b1b>:36: RuntimeWarning: invalid value encountered in sqrt
  lower = pred[r] - 1.96 * np.sqrt(var_c[r])
<ipython-input-146-fe9ebcac4b1b>:37: RuntimeWarning: invalid value encountered in sqrt
  upper = pred[r] + 1.96 * np.sqrt(var_c[r])
 65%|██████▌   | 13/20 [3:07:00<1:32:55, 796.46s/it]<ipython-input-146-fe9ebcac4b1b>:36: RuntimeWarning: invalid value encountered in sqrt
  lower = pred[r] - 1.96 * np.sqrt(var_c[r])
<ipython-input-146-fe9ebcac4b1b>:37: RuntimeWarning: invalid value encountered in sqrt
  upper = pred[r] + 1.96 * np.sqrt(var_c[r])
100%|██████████| 20/20 [4:35:27<00:00, 826.37s/it]  

In [147]:
result_100

{500: {'normal': [NormaltestResult(statistic=1.214322215467329, pvalue=0.5448955752443831),
   NormaltestResult(statistic=0.200314314039935, pvalue=0.9046952276571867),
   NormaltestResult(statistic=6.569738612857765, pvalue=0.03744547865694962),
   NormaltestResult(statistic=2.4029255015977844, pvalue=0.30075396190462905),
   NormaltestResult(statistic=36.16356746673378, pvalue=1.4033988025486531e-08),
   NormaltestResult(statistic=1.6130497498736662, pvalue=0.44640669291564306),
   NormaltestResult(statistic=0.2548225900254364, pvalue=0.8803715057139065),
   NormaltestResult(statistic=2.6105909530233706, pvalue=0.2710924217672585),
   NormaltestResult(statistic=1.1414813524833614, pvalue=0.5651067225287286),
   NormaltestResult(statistic=2.1417033702902732, pvalue=0.34271650652808583),
   NormaltestResult(statistic=2.568918595125577, pvalue=0.2768002097247502),
   NormaltestResult(statistic=0.9139709302102978, pvalue=0.6331895433393087),
   NormaltestResult(statistic=4.91475364778875

In [161]:
result = result_100

print("---------- B_n = 500 ----------")
B_n = 500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 1000 ----------")
B_n = 1000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 2500 ----------")
B_n = 2500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 5000 ----------")
B_n = 5000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

---------- B_n = 500 ----------
normality rejection rate: 0.1
mean variance ratio (corrected): 1.41488566664854
std variance ratio (corrected): 0.36177384041205496
mean variance ratio (uncorrected): 12.33788174593126
std variance ratio (uncorrected): 3.6435757620175067
mean coverage prob (corrected): 0.9433333333333334
std coverage prob (corrected): 0.04126098806139366
mean coverage prob (uncorrected): 1.0
std coverage prob (uncorrected): 0.0

---------- B_n = 1000 ----------
normality rejection rate: 0.15
mean variance ratio (corrected): 1.2520631140428555
std variance ratio (corrected): 0.33967333714044773
mean variance ratio (uncorrected): 7.119019728047933
std variance ratio (uncorrected): 2.0650916321811965
mean coverage prob (corrected): 0.9500000000000002
std coverage prob (corrected): 0.047596555534675324
mean coverage prob (uncorrected): 1.0
std coverage prob (uncorrected): 0.0

---------- B_n = 2500 ----------
normality rejection rate: 0.1
mean variance ratio (corrected): 1.0

## k = 250

In [148]:
result_250 = {500:  {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []},
              1000: {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []},
              2500: {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []},
              5000: {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []}}

rep = 45
incre = 1000
n_sub = 250

for _ in tqdm(range(20)):
    
    test = df_test.iloc[np.random.choice(730)].values[1:].reshape(1, -1)

    # print("========================= B = 500 =========================")

    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 500)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_250[500]["normal"].append(stats.normaltest(pred))
    result_250[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_250[500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_250[500]["cc_c"].append(count_c / rep)
    result_250[500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 1000 =========================")
    
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 1000)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        

    result_250[1000]["normal"].append(stats.normaltest(pred))
    result_250[1000]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_250[1000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_250[1000]["cc_c"].append(count_c / rep)
    result_250[1000]["cc_uc"].append(count_uc / rep)



    # print("========================= B = 2500 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 2500)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])


    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        

    result_250[2500]["normal"].append(stats.normaltest(pred))
    result_250[2500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_250[2500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_250[2500]["cc_c"].append(count_c / rep)
    result_250[2500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 5000 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 5000)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1      
            
    result_250[5000]["normal"].append(stats.normaltest(pred))
    result_250[5000]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_250[5000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_250[5000]["cc_c"].append(count_c / rep)
    result_250[5000]["cc_uc"].append(count_uc / rep)



100%|██████████| 20/20 [5:53:09<00:00, 1059.46s/it]  


In [121]:
result_250

{500: {'normal': [NormaltestResult(statistic=1.81894812051887, pvalue=0.4027359832009949),
   NormaltestResult(statistic=7.843641773185787, pvalue=0.019804999233850786),
   NormaltestResult(statistic=0.15178036724473495, pvalue=0.9269179917466557),
   NormaltestResult(statistic=0.1591336510798621, pvalue=0.923516303430897),
   NormaltestResult(statistic=3.1809216170667822, pvalue=0.20383166272006548),
   NormaltestResult(statistic=3.5927557958002927, pvalue=0.16589870331168724),
   NormaltestResult(statistic=6.1939597065067575, pvalue=0.04518546325593183),
   NormaltestResult(statistic=3.1713935666714184, pvalue=0.20480503865010558),
   NormaltestResult(statistic=2.512979282717559, pvalue=0.28465150358967795),
   NormaltestResult(statistic=0.10955706400168999, pvalue=0.9466947873387818)],
  'vr_c': [2.9349268562688637,
   2.6111449521962493,
   5.9218180172810255,
   2.7520194596194307,
   2.952271151971701,
   2.323188597192605,
   2.450901150499811,
   2.6391237401472103,
   1.950861

In [162]:
result = result_250

print("---------- B_n = 500 ----------")
B_n = 500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 1000 ----------")
B_n = 1000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 2500 ----------")
B_n = 2500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 5000 ----------")
B_n = 5000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

---------- B_n = 500 ----------
normality rejection rate: 0.0
mean variance ratio (corrected): 2.3187639994879445
std variance ratio (corrected): 0.5271080759753907
mean variance ratio (uncorrected): 13.813518470938075
std variance ratio (uncorrected): 3.4065431110895985
mean coverage prob (corrected): 0.9833333333333334
std coverage prob (corrected): 0.018425693279752218
mean coverage prob (uncorrected): 1.0
std coverage prob (uncorrected): 0.0

---------- B_n = 1000 ----------
normality rejection rate: 0.0
mean variance ratio (corrected): 1.6374497082604613
std variance ratio (corrected): 0.3878547541739054
mean variance ratio (uncorrected): 7.625984034133518
std variance ratio (uncorrected): 2.096280165985379
mean coverage prob (corrected): 0.968888888888889
std coverage prob (corrected): 0.03325917677132391
mean coverage prob (uncorrected): 1.0
std coverage prob (uncorrected): 0.0

---------- B_n = 2500 ----------
normality rejection rate: 0.05
mean variance ratio (corrected): 1.21

## k = 500

In [150]:
result_500 = {500:  {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []},
              1000: {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []},
              2500: {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []},
              5000: {"normal": [], "vr_c": [], "vr_uc": [], "cc_c": [], "cc_uc": []}}

rep = 45
incre = 1000
n_sub = 500

for _ in tqdm(range(20)):
    
    test = df_test.iloc[np.random.choice(730)].values[1:].reshape(1, -1)

    # print("========================= B = 500 =========================")

    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 500)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_500[500]["normal"].append(stats.normaltest(pred))
    result_500[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_500[500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_500[500]["cc_c"].append(count_c / rep)
    result_500[500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 1000 =========================")
    
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 1000)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        

    result_500[1000]["normal"].append(stats.normaltest(pred))
    result_500[1000]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_500[1000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_500[1000]["cc_c"].append(count_c / rep)
    result_500[1000]["cc_uc"].append(count_uc / rep)



    # print("========================= B = 2500 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 2500)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])


    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        

    result_500[2500]["normal"].append(stats.normaltest(pred))
    result_500[2500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_500[2500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_500[2500]["cc_c"].append(count_c / rep)
    result_500[2500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 5000 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 5000)
        model.fit(X, y, n_subsamples = n_sub, replace = True)
        pred.append(model.predict(test)[0])
        var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
    count_c = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_c[r])
        upper = pred[r] + 1.96 * np.sqrt(var_c[r])
        if target >= lower and target <= upper:
            count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1      
            
    result_500[5000]["normal"].append(stats.normaltest(pred))
    result_500[5000]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_500[5000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
    result_500[5000]["cc_c"].append(count_c / rep)
    result_500[5000]["cc_uc"].append(count_uc / rep)



100%|██████████| 20/20 [18:24:23<00:00, 3313.18s/it]  


In [151]:
result_500

{500: {'normal': [NormaltestResult(statistic=2.1471584455214345, pvalue=0.3417830080199248),
   NormaltestResult(statistic=0.24475474727790414, pvalue=0.8848143998612369),
   NormaltestResult(statistic=6.030512312027104, pvalue=0.04903327371027308),
   NormaltestResult(statistic=0.17417159690664152, pvalue=0.9165984495300576),
   NormaltestResult(statistic=6.331212253355474, pvalue=0.04218856244952592),
   NormaltestResult(statistic=1.5409731238064017, pvalue=0.46278783859004813),
   NormaltestResult(statistic=0.9276543696095872, pvalue=0.6288722237530839),
   NormaltestResult(statistic=1.6270338692838915, pvalue=0.44329627744341493),
   NormaltestResult(statistic=1.305167369092897, pvalue=0.5206987160943239),
   NormaltestResult(statistic=4.143039986930691, pvalue=0.12599412583616157),
   NormaltestResult(statistic=0.1896940381576052, pvalue=0.9095120618191022),
   NormaltestResult(statistic=0.9969158867267427, pvalue=0.6074666858589433),
   NormaltestResult(statistic=10.1486878973881

In [163]:
result = result_500

print("---------- B_n = 500 ----------")
B_n = 500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 1000 ----------")
B_n = 1000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 2500 ----------")
B_n = 2500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 5000 ----------")
B_n = 5000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

---------- B_n = 500 ----------
normality rejection rate: 0.15
mean variance ratio (corrected): 5.080109077635122
std variance ratio (corrected): 1.9789373058262056
mean variance ratio (uncorrected): 20.730640616365893
std variance ratio (uncorrected): 8.752893951913178
mean coverage prob (corrected): 0.9966666666666667
std coverage prob (corrected): 0.014529663145135577
mean coverage prob (uncorrected): 1.0
std coverage prob (uncorrected): 0.0

---------- B_n = 1000 ----------
normality rejection rate: 0.05
mean variance ratio (corrected): 3.0846850599910836
std variance ratio (corrected): 1.2335681846767959
mean variance ratio (uncorrected): 11.073088665724816
std variance ratio (uncorrected): 4.871034180409476
mean coverage prob (corrected): 0.9866666666666667
std coverage prob (corrected): 0.024745619390355664
mean coverage prob (uncorrected): 1.0
std coverage prob (uncorrected): 0.0

---------- B_n = 2500 ----------
normality rejection rate: 0.05
mean variance ratio (corrected): 1

In [30]:
X = df[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
y = df[["RMSD"]].values.ravel()

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [47]:
regr = RandomForestRegressor()
regr.fit(X_train[:3000], y_train[:3000])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [48]:
np.mean((regr.predict(X_test) - y_test) ** 2)

21.76038640147168

##### 